## Import Modules

In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import (year, month, dayofmonth, hour,
                                   weekofyear, dayofweek, date_format,
                                   monotonically_increasing_id)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1663609079191_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## AWS Credentials

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ["AWS_ACCESS_KEY_ID"]= ''
os.environ["AWS_SECRET_ACCESS_KEY"]= ''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://jk-loaded-data/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Create Spark Session

In [4]:

"""Create a apache spark session."""
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
    .getOrCreate()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Process Song data

In [5]:
# get filepath to song data file
song_data = input_data + 'song_data/*/*/*/*.json'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Extract Song table & Create parquet file

In [7]:
# read song data file
song_df = spark.read.json(song_data)

# extract columns to create songs table
# songs_table = song_df.select('song_id', 'title', 'artist_id','year', 'duration').dropDuplicates()
songs_table = song_df.select(["song_id", "title", "artist_id", "year", "duration"]).distinct()
songs_table.createOrReplaceTempView('songs')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# write songs table to parquet files partitioned by year and artist
songs_table.write.partitionBy('year', 'artist_id') \
                 .parquet(os.path.join(output_data, 'songs/songs.parquet'), 'overwrite')

In [9]:
songs_table.show(5, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----------------------------------------------------+------------------+----+---------+
|song_id           |title                                               |artist_id         |year|duration |
+------------------+----------------------------------------------------+------------------+----+---------+
|SOHQZIB12A6D4F9FAF|N****_ What's Up [Featuring 50 Cent] (Album Version)|ARWAFY51187FB5C4EF|2006|196.85832|
|SOUCTAJ12AB018F4F4|Laat het licht maar uit (we doen het in 't donker)  |ARPZTBN12086C1563D|2008|222.06649|
|SOQWBCS12A58A7C2C3|Train Track                                         |ARV62991187B99DE55|2007|168.64608|
|SOVEHRU12AB017CEE6|Tomorrow We Sail (Jazz)                             |AR2LRXK1187FB593DD|0   |324.85832|
|SONWUEN12AB018329C|Welcome (instr)                                     |AR6OQLJ1187B9B032E|0   |189.17832|
+------------------+----------------------------------------------------+------------------+----+---------+
only showing top 5 rows

## Extract Artists table & Create parquet file

In [10]:
# extract columns to create artists table
artists_table = song_df.select('artist_id', 'artist_name', 'artist_location',
                          'artist_latitude', 'artist_longitude') \
                  .withColumnRenamed('artist_name', 'name') \
                  .withColumnRenamed('artist_location', 'location') \
                  .withColumnRenamed('artist_latitude', 'latitude') \
                  .withColumnRenamed('artist_longitude', 'longitude') \
                  .dropDuplicates()
artists_table.createOrReplaceTempView('artists')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# write artists table to parquet files
# artists_table.write.mode("overwrite")
artists_table.write.mode("overwrite").parquet('artists/artists.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
artists_table.show(5, truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+--------------------+--------+----------+
|artist_id         |name             |location            |latitude|longitude |
+------------------+-----------------+--------------------+--------+----------+
|ARQSM561187FB4A0CF|Jack's Mannequin |Orange County , USA |33.6671 |-117.76505|
|ARKC83D1187B9AB367|Mark Lowry       |                    |null    |null      |
|AR6A42V1187FB47B61|The Apathy Eulogy|                    |null    |null      |
|ARNJ7441187B999AFD|Serge Gainsbourg |Paris, France       |48.85692|2.34121   |
|ARG4K2W1187B99EA5D|Charles Aznavour |Virginia            |38.00335|-79.77127 |
+------------------+-----------------+--------------------+--------+----------+
only showing top 5 rows

## Process log Data

In [20]:
# get filepath to log data file
log_data = input_data + 'log_data/*/*/'

# read log data file
log_df = spark.read.json(log_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
log_df.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|           song|status|           ts|           userAgent|userId|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|  Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|The Prodigy|Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|The Big Gundown|

In [22]:
# filter by actions for song plays
actions_df = log_df.filter(log_df.page == 'NextSong') \
               .select('ts', 'userId', 'level', 'song', 'artist',
                       'sessionId', 'location', 'userAgent')
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: str(int(int(x)/1000)))
actions_df = actions_df.withColumn('timestamp', get_timestamp(actions_df.ts))


# create datetime column from original timestamp column
get_datetime = udf(lambda x: str(datetime.fromtimestamp(int(x) / 1000)))
actions_df = actions_df.withColumn('datetime', get_datetime(actions_df.ts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# extract columns for users table    
users_table = log_df.select('userId', 'firstName', 'lastName',
                        'gender', 'level').dropDuplicates()
users_table.createOrReplaceTempView('users')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# write users table to parquet files
users_table.write.mode("overwrite").parquet('users/users.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
users_table.show(5, truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|57    |Katherine|Gay     |F     |free |
|84    |Shakira  |Hunt    |F     |free |
|22    |Sean     |Wilson  |F     |free |
|52    |Theodore |Smith   |M     |free |
|80    |Tegan    |Levine  |F     |paid |
+------+---------+--------+------+-----+
only showing top 5 rows

In [27]:
# extract columns to create time table
time_table = actions_df.select('datetime') \
                       .withColumn('start_time', actions_df.datetime) \
                       .withColumn('hour', hour('datetime')) \
                       .withColumn('day', dayofmonth('datetime')) \
                       .withColumn('week', weekofyear('datetime')) \
                       .withColumn('month', month('datetime')) \
                       .withColumn('year', year('datetime')) \
                       .withColumn('weekday', dayofweek('datetime')) \
                       .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# write time table to parquet files partitioned by year and month
time_table.write.partitionBy('year', 'month').mode("overwrite") \
                .parquet('time/time.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# read in song data to use for songplays table
song_df = spark.read.json(input_data + 'song_data/*/*/*/*.json')

# extract columns from joined song and log datasets to create songplays table
actions_df = actions_df.alias('log_df')
song_df = song_df.alias('song_df')
joined_df = actions_df.join(song_df, col('log_df.artist') == col(
    'song_df.artist_name'), 'inner')
songplays_table = joined_df.select(
    col('log_df.datetime').alias('start_time'),
    col('log_df.userId').alias('user_id'),
    col('log_df.level').alias('level'),
    col('song_df.song_id').alias('song_id'),
    col('song_df.artist_id').alias('artist_id'),
    col('log_df.sessionId').alias('session_id'),
    col('log_df.location').alias('location'), 
    col('log_df.userAgent').alias('user_agent'),
    year('log_df.datetime').alias('year'),
    month('log_df.datetime').alias('month')) \
    .withColumn('songplay_id', monotonically_increasing_id())

songplays_table.createOrReplaceTempView('songplays')
# write songplays table to parquet files partitioned by year and month
time_table = time_table.alias('timetable')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
songplays_table.write.partitionBy('year', 'month').mode("overwrite") \
.parquet('songplays/songplays.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
songplays_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+-----------+
|          start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|year|month|songplay_id|
+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+-----------+
|2018-11-17 14:13:...|     73| paid|SOVIYJY12AF72A4B00|AR4T2IF1187B9ADBB7|       518|Tampa-St. Petersb...|"Mozilla/5.0 (Mac...|2018|   11|          0|
|2018-11-28 09:46:...|     82| paid|SOVIYJY12AF72A4B00|AR4T2IF1187B9ADBB7|       140|Atlanta-Sandy Spr...|"Mozilla/5.0 (Mac...|2018|   11|          1|
|2018-11-15 19:38:...|     97| paid|SOVIYJY12AF72A4B00|AR4T2IF1187B9ADBB7|       605|Lansing-East Lans...|"Mozilla/5.0 (X11...|2018|   11|          2|
|2018-11-17 14:13:...|     73| paid|SOVYXYL12AF72A3373|AR4T2IF1187B9ADBB7|       518|Tampa-St.